In [1]:
import pandas as pd
import numpy as np

from ast import literal_eval

import statsmodels.formula.api as smf

import setup_modules
from lib.fomc import get_lname

In [2]:
fomc = pd.read_csv('../working-csvs/fomc.csv', index_col=0)
fred = pd.read_csv('../working-csvs/fred.csv')

In [17]:
mfdff = pd.read_csv('../working-csvs/mfdff_selected.csv', index_col=0)
mfdff['final_norm_svect'] = mfdff['final_norm_svect'].apply(literal_eval)
mfdff['final_diff_exp_norm'] = mfdff['final_diff_exp_norm'].apply(literal_eval)

In [18]:
mfdff.columns

Index(['date_x', 'section', 'lname', 'svect', 'voter', 'sent', 'region',
       'female', 'chair', 'exp', 'exp_tertile', 'expd', 'date_section_mean',
       'date_section_std', 'norm_svect', 'temp_svect', 'temp_total',
       'total_exp', 'total_expd', 'num_expd', 'avg_expd', 'diff_exp_norm',
       'use', 'date', 'year', 'm', 'tprob_0', 'tprob_1', 'tprob_2', 'tprob_3',
       'tprob_4', 'tprob_5', 'tprob_6', 'tprob_7', 'tprob_8', 'tprob_9',
       'tprob_10', 'tprob_11', 'tprob_12', 'tprob_13', 'tprob_14', 'tprob_15',
       'tprob_16', 'tprob_17', 'tprob_18', 'tprob_19', 'tprob_20', 'tprob_21',
       'tprob_22', 'tprob_23', 'tprob_24', 'tprob_25', 'tprob_26', 'tprob_27',
       'tprob_28', 'tprob_29', 'tprob_30', 'tprob_31', 'tprob_32', 'tprob_33',
       'tprob_34', 'tprob_35', 'tprob_36', 'tprob_37', 'tprob_38', 'tprob_39',
       'tprob_40', 'tprob_41', 'tprob_42', 'tprob_43', 'tprob_44',
       'final_norm_svect', 'final_diff_exp_norm'],
      dtype='object')

In [19]:
mfdff['dist'] = mfdff['final_norm_svect'].map(lambda x : np.sum(np.abs(x)))
mfdff['dist_exp'] = mfdff['final_diff_exp_norm'].map(lambda x: np.sum(np.abs(x)))

In [20]:
mfdff['date'] = pd.to_datetime(mfdff['date'])
mfdff['merge_date'] = mfdff['date'].dt.strftime('%Y-%m')
mfdff.drop(columns='date', inplace=True)

In [21]:
fred = fred.rename(columns={'PCE_PC1': 'pceinf', 'PCEPILFE_PC1': 'corepceinf', 'UNRATE': 'unrate', 'DATE': 'date'})

In [22]:
fred

,date,unrate,pceinf,corepceinf,merge_date
0,1994-01-01,6.6,5.63043,2.24661,1994-01
1,1994-02-01,6.6,6.36696,2.27283,1994-02
2,1994-03-01,6.5,6.74325,2.39726,1994-03
3,1994-04-01,6.4,6.32084,2.25372,1994-04
4,1994-05-01,6.1,5.58746,2.13570,1994-05
...,...,...,...,...,...
356,2023-09-01,3.8,5.54651,3.59248,2023-09
357,2023-10-01,3.8,4.90982,3.37936,2023-10
358,2023-11-01,3.7,5.45089,3.15332,2023-11
359,2023-12-01,3.7,5.89681,2.93151,2023-12


In [23]:
fred['date'] = pd.to_datetime(fred['date'])
fred['merge_date'] = fred['date'].dt.strftime('%Y-%m')

In [24]:
df = mfdff.merge(fred, how='left', on='merge_date')

In [25]:
panel = df[(df['section'] == 2) & (df['use'] == True)].copy()

In [26]:
panel

,date_x,section,lname,svect,voter,sent,region,female,chair,exp,...,tprob_44,final_norm_svect,final_diff_exp_norm,dist,dist_exp,merge_date,date,unrate,pceinf,corepceinf
15,1994-02-04,2,boehne,[ 0. 0.39055794 -1. 0. ...,0,3,Philadelphia,0.0,Greenspan,13.016438,...,-0.311440,"[0.3846263, 2.6291501, -3.116221, 0.0, 0.0, 3....","[0.1322509, 2.6103134, -2.8756371, 0.0, 0.0, 3...",18.811015,20.093750,1994-02,1994-02-01,6.6,6.36696,2.27283
18,1994-02-04,2,greenspan,[-0.29740135 -0.32435035 0. 0. ...,1,30,governor,0.0,Greenspan,6.490411,...,1.768628,"[-0.6733815, -2.2524156, 0.170221, 0.0, 0.0, -...","[-0.9257569, -2.2712524, 0.4108053, 0.0, 0.0, ...",37.140997,36.653184,1994-02,1994-02-01,6.6,6.36696,2.27283
22,1994-02-04,2,lindsey,[-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1,4,governor,0.0,Greenspan,2.194521,...,1.067303,"[-3.172882, -0.0376734, 0.170221, 0.0, 0.0, -0...","[-3.425257, -0.0565101, 0.4108053, 0.0, 0.0, -...",11.269138,14.290676,1994-02,1994-02-01,6.6,6.36696,2.27283
24,1994-02-04,2,melzer,[ 0. 0. 0. 0. ...,0,5,St. Louis,0.0,Greenspan,8.684932,...,-0.210439,"[0.3846263, -0.0376734, 0.170221, 0.0, 0.0, -0...","[0.1322509, -0.0565101, 0.4108053, 0.0, 0.0, -...",14.389721,16.383390,1994-02,1994-02-01,6.6,6.36696,2.27283
44,1994-03-22,2,greenspan,[ 0. 0.51533743 1.39780922 0. ...,1,36,governor,0.0,Greenspan,6.616438,...,0.004804,"[0.3162277, 2.174603, 2.321346, 0.3162277, -0....","[0.4969293, 2.2678999, 2.1472607, 0.0, 0.0, -1...",46.722515,44.209175,1994-03,1994-03-01,6.5,6.74325,2.39726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5879,2018-12-19,2,kaplan,[ 0. 0. 0. 0. ...,0,3,Dallas,0.0,Powell,3.282192,...,-0.451435,"[0.0485393, 0.0, -0.3889801, 0.080885, 0.0, 0....","[0.0, 0.0, -1.85493179, -2.47183336, 0.0, 5.88...",12.124023,102.443993,2018-12,2018-12-01,3.9,2.82085,2.03808
5880,2018-12-19,2,kashkari,[-0.39757576 0. 0. 0. ...,0,9,Minneapolis,0.0,Powell,2.967123,...,-0.451435,"[-2.7999033, 0.0, -0.3889801, -2.9480877, 0.0,...","[-2.84844268, 0.0, -1.85493179, -3.27615621, 0...",23.595003,94.930528,2018-12,2018-12-01,3.9,2.82085,2.03808
5881,2018-12-19,2,mester,[ 0. 0. 0. 0. ...,1,9,Cleveland,1.0,Powell,4.553425,...,0.733341,"[0.0485393, 0.0, -0.3889801, 0.080885, 0.0, 0....","[0.0, 0.0, -0.1854931, -0.2471833, 0.0, 0.5886...",18.948209,17.709145,2018-12,2018-12-01,3.9,2.82085,2.03808
5883,2018-12-19,2,quarles,[ 0. 0. 0. 0. ...,1,5,governor,0.0,Powell,1.254795,...,-4.514350,"[4.85393099, 0.0, -3.88980189, 8.08850907, 0.0...","[0.0, 0.0, -1.85493179, -2.47183336, 0.0, 5.88...",112.893198,95.407835,2018-12,2018-12-01,3.9,2.82085,2.03808


In [27]:
panel['pceinf'] = panel['pceinf'].astype('float')
panel['corepceinf'] = panel['corepceinf'].astype('float')

In [39]:
cov_name_map = {'exp': 'Experience', 'unrate': 'Unemployment',
                'I(exp ** 2)': 'Experience$^2$', 'pceinf': 'PCE inflation', 'C(female)[T.1.0]': 'Female', 'C(voter)[T.1]': 'Voter', 'I(exp ** 2):pceinf': 'Experience$^2$ $\\times$ PCE inflation',
                'I(exp ** 2):unrate': 'Experience$^2$ $\\times$ Unemployment', 'exp:pceinf': 'Experience $\\times$ PCE Inflation', 'exp:unrate': 'Experience $\\times$ Unemployment',
                "C(chair, Treatment(reference='Greenspan'))[T.Yellen]": "Yellen", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]": 'Bernanke', "C(chair, Treatment(reference='Greenspan'))[T.Powell]": "Powell"
                }

In [42]:

model1 = smf.ols("dist ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})
model2 = smf.ols("dist ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the Euclidean distance of the stance vectors of individual committee participants from the committee average in the monetary policy discussion section. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1]", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('table1.tex', 'w') as f:
    f.write(sg.render_latex())
sg

In [35]:
panel['expd']

15       True
18       True
22      False
24       True
44       True
        ...  
5879    False
5880    False
5881     True
5883    False
5884     True
Name: expd, Length: 1738, dtype: bool

In [37]:
inexp_panel = panel[panel['expd'] == False]

In [43]:
model1 = smf.ols("dist_exp ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})
model2 = smf.ols("dist_exp ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the Euclidean distance of the stance vectors of individual committee participants from the committee average in the monetary policy discussion section. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance of Experienced Members')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1]", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('table2.tex', 'w') as f:
    f.write(sg.render_latex())
sg